In [83]:
import pandas as pd
import numpy as np
import random

In [107]:
class Simulator():
    """
    Class to generate fake data
    TODO: 
        figure out seed setting
        generate correlated distributions
        generate time series
        linear / non linear combination of variables
        add a UI?
    """
    
    def __init__(self, num_rows: int, seed = 42):
        """
        """
        
        self.num_rows = num_rows
        self.seed = np.random.seed(seed)
    
    def generate_categorical_column(self, num_categories: int, distribution = 'uniform'):
        """
        Returns a Pandas Series with num_rows
        """
        return pd.Series(data = self.generate_categorical_distribution(num_categories), index = range(self.num_rows), dtype= 'object')
    
    def generate_categorical_distribution(self, num_categories: int, name: str = 'color', distribution = 'uniform'):
        """
        Specify a categorical feature's distribution
        By default the distribution is uniform - all categories will have an equal chance of being drawn
        User can also pass a size [num_categories] array of probabilities for each category
        """

        categories = [name + "_" + str(i+1) for i in range(4)]
        if distribution == 'uniform':
            np.random.seed(self.seed)
            categorical_column = np.random.choice(categories, size = self.num_rows)

        elif type(distribution) == list or type(distribution) == np.array:
            categorical_column = np.random.choice(categories, size = self.num_rows, p = distribution)
        
        return categorical_column
    
    def generate_numerical_column(self, distribution = 'normal'):
        """
        TODO: different distribution types should be selectable (normal, uniform, exponential +)
        """






In [108]:
a = Simulator(num_rows = 100)

In [109]:
a.generate_categorical_column(num_categories = 4, distribution = [0.25, 0.3, 0.25, 0.2])

0     color_2
1     color_3
2     color_1
3     color_1
4     color_1
       ...   
95    color_2
96    color_1
97    color_3
98    color_1
99    color_1
Length: 100, dtype: object